In [11]:
import psycopg2
import json
import numpy as np

#importing local modules
from tokenize_sentences2db import openai_embeddings,log
import importlib
#importlib.reload(chunk_splitter)
#importlib.reload(openai_embeddings)

# Function to perform the search
def search(query, model, db_config,number_of_results):
    # Encode the query
    log("Encoding the query...")
    query_embedding = openai_embeddings(model,query)[0]
    log("Finding the most similar projects...")
    # Connect to the database
    with psycopg2.connect(**db_config) as conn:
        c = conn.cursor()

        # Search for the top projects by cosine similarity
        c.execute("""
            SELECT project_id, chunk, embedding <-> %s::VECTOR AS distance
            FROM embeddings_openai
            ORDER BY distance ASC
            LIMIT %s;
        """, (list(query_embedding), number_of_results))
        
        results = c.fetchall()
    log("Done! Found {} results.".format(len(results)))
    return results


# Define the database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}



In [12]:
#convert this notebook to a python script
!jupyter nbconvert --to script search.ipynb

[NbConvertApp] Converting notebook search.ipynb to script
[NbConvertApp] Writing 2181 bytes to search.py


In [16]:
model="text-embedding-ada-002"
number_of_results = 5

# Database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}

# Load the projects
with open("digital_agriculture_projects.json", "r") as f:
    projects = json.load(f)

# Define the query
query = "Satellite Remote sensing"

# Perform the search
results = search(query, model, db_config,number_of_results)
results=results[::-1]
# Print the results
for project_id, chunk, distance in results:
    project = next((p for p in projects if ",".join(p["ids"]) == project_id), None)
    print(f"Project ID: {project_id}")
    print(f"Project title: {project['title']}")
    print(f"Project abstract: {project['abstract']}")
    print(f"Relevant snippet: {chunk}")
    print(f"Distance: {distance}")
    print("\n")


17:06:46 - Encoding the query...
17:06:56 - Finding the most similar projects...
17:06:56 - Done! Found 5 results.
Project ID: P164448
Project title: Myanmar - National Food and Agriculture System Project
Project abstract: The objective of the National Food and Agriculture System Project for Myanmar is to increase agricultural productivity and diversification and to enhance market access for Selected Value Chains in the Project Area, and respond to an Eligible Crisis or Emergency. The Project has four components. 1. Agriculture Productivity Enhancement and Diversification component will focus on: (a) Strengthening Agricultural Research and Development System; (b) Improving the Quality and Utilization of Agricultural Inputs; (c) Strengthening Agriculture Extension Services through Digital Technologies and (d) Improving Irrigation and Drainage Infrastructure. 2. Value-Chain Development component will focus on: (a) Supporting Value Addition and Market Access and (b) International Standard